In [1]:
import pickle
from diffusion.sampling import *
form diffusion.likelihood import mmff_energy
import py3Dmol
from rdkit import Chem 
from ipywidgets import interact, Dropdown

/home/mila/l/lena-nehale.ezzine/.conda/envs/torsional_diffusion/lib/python3.9/site-packages/torch_geometric/typing.py:99: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /home/mila/l/lena-nehale.ezzine/.conda/envs/torsional_diffusion/lib/python3.9/site-packages/torch_spline_conv/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(
/home/mila/l/lena-nehale.ezzine/.conda/envs/torsional_diffusion/lib/python3.9/site-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /home/mila/l/lena-nehale.ezzine/.conda/envs/torsional_diffusion/lib/python3.9/site-packages/torch_sparse/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "
/home/mila/l/lena-nehale.ezzine/.conda/envs/torsional_diffusion/lib/python3.9/site-packages/torchani/aev.py:

In [34]:
# load conformers_20steps.pkl
conformers = pickle.load(open('conformers_20steps.pkl', 'rb'))
smis = list(conformers.keys())

In [35]:
idx = 0
mols = conformers[smis[idx]]

In [42]:
for conf_id, mol in enumerate(mols):
    mol_block = Chem.MolToMolBlock(mol)
    viewer = py3Dmol.view(width=400, height=400)
    viewer.addModel(mol_block, "mol")

In [43]:
from ipywidgets import interact, IntSlider
viewer.setStyle({'stick': {}})
viewer.zoomTo()
def update_frame(frame):
    viewer.setFrame(frame)
    viewer.show()
    viewer.render()

interact(update_frame, frame=IntSlider(min=0, max=len(mols)-1, step=1, value=0))


#viewer.animate({'loop': 'forward', 'reps': 0})
#viewer.show()

interactive(children=(IntSlider(value=0, description='frame', max=137), Output()), _dom_classes=('widget-inter…

<function __main__.update_frame(frame)>

In [ ]:
mmff_energy(mols[0])